In [4]:
pip install pyomo

   ---------------------------------------- 0.0/3.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.7 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.7 MB ? eta -:--:--
   -------- ------------------------------- 0.8/3.7 MB 1.6 MB/s eta 0:00:02
   -------------- ------------------------- 1.3/3.7 MB 1.9 MB/s eta 0:00:02
   ------------------- -------------------- 1.8/3.7 MB 2.2 MB/s eta 0:00:01
   ---------------------------- ----------- 2.6/3.7 MB 2.3 MB/s eta 0:00:01
   ------------------------------------ --- 3.4/3.7 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 3.7/3.7 MB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
from pyomo.environ import *

# Parameters
numOperations = 3
numJobs = 2
numMachines = 3
numMaintCycles = 3
numPositions = 2
B = 100000

# Model
model = ConcreteModel()

# Sets
model.I = RangeSet(1, numOperations)  # Operations
model.J = RangeSet(1, numJobs)        # Jobs
model.M = RangeSet(1, numMachines)    # Machines
model.W = RangeSet(1, numMaintCycles) # Maintenance cycles
model.R = RangeSet(1, numPositions)   # Positions

In [7]:
# 
pijm = {
    (1, 1, 1): 3, (1, 1, 2): 0, (1, 1, 3): 0,
    (2, 1, 1): 0, (2, 1, 2): 0, (2, 1, 3): 4,
    (3, 1, 1): 0, (3, 1, 2): 0, (3, 1, 3): 0,
    (1, 2, 1): 4, (1, 2, 2): 0, (1, 2, 3): 0,
    (2, 2, 1): 0, (2, 2, 2): 2, (2, 2, 3): 0,
    (3, 2, 1): 0, (3, 2, 2): 0, (3, 2, 3): 6,
}

STijm = {
    (1, 1, 1): 1, (1, 1, 2): 0, (1, 1, 3): 0,
    (2, 1, 1): 0, (2, 1, 2): 0, (2, 1, 3): 2,
    (3, 1, 1): 0, (3, 1, 2): 0, (3, 1, 3): 0,
    (1, 2, 1): 2, (1, 2, 2): 0, (1, 2, 3): 0,
    (2, 2, 1): 0, (2, 2, 2): 1, (2, 2, 3): 0,
    (3, 2, 1): 0, (3, 2, 2): 0, (3, 2, 3): 1,
}

fijm = {
    (1, 1, 1): 1, (1, 1, 2): 0, (1, 1, 3): 0,
    (2, 1, 1): 0, (2, 1, 2): 0, (2, 1, 3): 1,
    (3, 1, 1): 0, (3, 1, 2): 0, (3, 1, 3): 0,
    (1, 2, 1): 1, (1, 2, 2): 0, (1, 2, 3): 0,
    (2, 2, 1): 0, (2, 2, 2): 1, (2, 2, 3): 0,
    (3, 2, 1): 0, (3, 2, 2): 0, (3, 2, 3): 1,
}

In [8]:
# Decision variables
model.sij = Var(model.I, model.J, within=NonNegativeReals)  # Start time of operation i of job j
model.cij = Var(model.I, model.J, within=NonNegativeReals)  # Completion time of operation i of job j
model.cirm = Var(model.I, model.R, model.M, within=NonNegativeReals)
model.sirm = Var(model.I, model.R, model.M, within=NonNegativeReals)
model.yijrm = Var(model.I, model.J, model.R, model.M, within=Binary)
model.Cmax = Var(within=NonNegativeReals)

In [9]:
model.obj = Objective(expr=model.Cmax, sense=minimize)

model.constraints = ConstraintList()

In [10]:
for j in model.J:
    for i in model.I:
        model.constraints.add(model.Cmax >= model.cij[i, j])

In [11]:
for m in model.M:
    for j in model.J:
        for r in model.R:
            for i in model.I:
                model.constraints.add(model.cij[i, j] >= model.cirm[i, r, m] - B * (1 - model.yijrm[i, j, r, m]))


In [12]:
for m in model.M:
    for r in model.R:
        for i in model.I:
            model.constraints.add(
                model.cirm[i, r, m] == model.sirm[i, r, m] +
                sum(pijm[i, j, m] + STijm[i, j, m] * model.yijrm[i, j, r, m] for j in model.J)
            )

In [13]:
for m in model.M:
    for j in model.J:
        for i in model.I:
            model.constraints.add(sum(model.yijrm[i, j, r, m] for r in model.R) <= fijm[i, j, m])


In [14]:
for j in model.J:
    for i in model.I:
        model.constraints.add(
            sum(model.yijrm[i, j, r, m] * fijm[i, j, m] for r in model.R for m in model.M) == 1
        )


In [15]:
for m in model.M:
    for r in model.R:
        for j in model.J:
            for i in model.I:
                model.constraints.add(
                    model.sij[i, j] >= model.sirm[i, r, m] - B * (1 - model.yijrm[i, j, r, m])
                )


In [16]:
for m in model.M:
    for r in range(1, numPositions):
        for i in model.I:
            model.constraints.add(model.sirm[i, r + 1, m] >= model.cirm[i, r, m])


In [ ]:
for j in model.J:
    for i in range(1, numOperations):
        model.constraints.add(model.sij[i + 1, j] >= model.cij[i, j])

solver = SolverFactory('glpk')
solver.solve(model)

In [18]:
# Print results
print(f"Objective (Cmax): {model.Cmax.value}")
for i in model.I:
    for j in model.J:
        print(f"Operation {i}, Job {j}: Start = {model.sij[i, j].value}, Completion = {model.cij[i, j].value}")

Objective (Cmax): None
Operation 1, Job 1: Start = None, Completion = None
Operation 1, Job 2: Start = None, Completion = None
Operation 2, Job 1: Start = None, Completion = None
Operation 2, Job 2: Start = None, Completion = None
Operation 3, Job 1: Start = None, Completion = None
Operation 3, Job 2: Start = None, Completion = None


In [ ]:
import pandas as pd


fijm_data = {
    "Job": [1, 1, 1, 2, 2, 2],
    "Operation": [1, 2, 3, 1, 2, 3],
    "m=1": [1, 0, 0, 1, 0, 0],
    "m=2": [0, 0, 0, 0, 1, 0],
    "m=3": [0, 1, 0, 0, 0, 1]
}
fijm_df = pd.DataFrame(fijm_data)

pijm_data = {
    "Job": [1, 1, 2],
    "Operation": [1, 2, 3],
    "m=1": [3, None, None],
    "m=2": [None, None, None],
    "m=3": [None, 4, 6]
}
pijm_df = pd.DataFrame(pijm_data)

STijm_data = {
    "Job": [1, 1, 2],
    "Operation": [1, 2, 3],
    "m=1": [1, None, None],
    "m=2": [None, None, None],
    "m=3": [None, 2, 1]
}
STijm_df = pd.DataFrame(STijm_data)

dmw_data = {
    "Maintenance Activity": [1],
    "m=1": [0],
    "m=2": [0],
    "m=3": [3]
}
dmw_df = pd.DataFrame(dmw_data)

Q_data = {
    "Maximum number of maintenance activities": [1]
}
Q_df = pd.DataFrame(Q_data)

scenarios_data = {
    "Scenario": [1, 2, 3, 4, 5],
    "Machine 1 Sequence": ["O11 - O12", "O11 - O12", "O11 - O12", "O12 - O11", "O12 - O11"],
    "Machine 2 Sequence": ["O22", "O22", "O22", "O22", "O22"],
    "Machine 3 Sequence": ["O21 - PM - O23", "O21 - O23 - PM", "O23 - O21 - PM", "O21 - O23 - PM", "O21 - O23 - PM"],
    "Makespan": [20, 23, 29, 28, 25]
}
scenarios_df = pd.DataFrame(scenarios_data)

file_path = "complete_data.xlsx"
with pd.ExcelWriter(file_path) as writer:
    fijm_df.to_excel(writer, sheet_name="fijm", index=False)
    pijm_df.to_excel(writer, sheet_name="pijm", index=False)
    STijm_df.to_excel(writer, sheet_name="STijm", index=False)
    dmw_df.to_excel(writer, sheet_name="dmw", index=False)
    Q_df.to_excel(writer, sheet_name="Q", index=False)
    scenarios_df.to_excel(writer, sheet_name="Scenarios", index=False)



In [ ]:
input_data = pd.read_excel("complete_data.xlsx", sheet_name=None)

fijm = {(row['Job'], row['Operation'], m): row[m] for _, row in input_data['fijm'].iterrows() for m in ['m=1', 'm=2', 'm=3']}
pijm = {(row['Job'], row['Operation'], m): row[m] for _, row in input_data['pijm'].iterrows() for m in ['m=1', 'm=2', 'm=3']}
STijm = {(row['Job'], row['Operation'], m): row[m] for _, row in input_data['STijm'].iterrows() for m in ['m=1', 'm=2', 'm=3']}
dmw = {(1, int(m.split('=')[1])): row[m] for _, row in input_data['dmw'].iterrows() for m in ['m=1', 'm=2', 'm=3']}
Q = int(input_data['Q']['Maximum number of maintenance activities'][0])
scenarios = input_data['Scenarios'].to_dict('records')

print("Input binary parameters fijm")
for key, value in fijm.items():
    print(f"Job {key[0]}, Operation {key[1]}, Machine {key[2]}: {value}")

print("\nInput parameters of processing time pijm")
for key, value in pijm.items():
    print(f"Job {key[0]}, Operation {key[1]}, Machine {key[2]}: {value}")

print("\nInput parameters of setup time STijm")
for key, value in STijm.items():
    print(f"Job {key[0]}, Operation {key[1]}, Machine {key[2]}: {value}")

print("\nInput parameters of maintenance duration dmw")
for key, value in dmw.items():
    print(f"Machine {key[0]}, Maintenance Activity {key[1]}: {value}")

print("\nMaximum number of maintenance activities")
print(f"Q: {Q}")

print("\nResults for each scenario")
for scenario in scenarios:
    print(f"Scenario {scenario['Scenario']}: Makespan {scenario['Makespan']}")

Input binary parameters fijm
Job 1, Operation 1, Machine m=1: 1
Job 1, Operation 1, Machine m=2: 0
Job 1, Operation 1, Machine m=3: 0
Job 1, Operation 2, Machine m=1: 0
Job 1, Operation 2, Machine m=2: 0
Job 1, Operation 2, Machine m=3: 1
Job 1, Operation 3, Machine m=1: 0
Job 1, Operation 3, Machine m=2: 0
Job 1, Operation 3, Machine m=3: 0
Job 2, Operation 1, Machine m=1: 1
Job 2, Operation 1, Machine m=2: 0
Job 2, Operation 1, Machine m=3: 0
Job 2, Operation 2, Machine m=1: 0
Job 2, Operation 2, Machine m=2: 1
Job 2, Operation 2, Machine m=3: 0
Job 2, Operation 3, Machine m=1: 0
Job 2, Operation 3, Machine m=2: 0
Job 2, Operation 3, Machine m=3: 1

Input parameters of processing time pijm
Job 1.0, Operation 1.0, Machine m=1: 3.0
Job 1.0, Operation 1.0, Machine m=2: nan
Job 1.0, Operation 1.0, Machine m=3: nan
Job 1.0, Operation 2.0, Machine m=1: nan
Job 1.0, Operation 2.0, Machine m=2: nan
Job 1.0, Operation 2.0, Machine m=3: 4.0
Job 2.0, Operation 3.0, Machine m=1: nan
Job 2.0, Ope

In [2]:

fijm = {
    (1, 1, 1): 1, (1, 1, 2): 0, (1, 1, 3): 0,
    (1, 2, 1): 0, (1, 2, 2): 0, (1, 2, 3): 1,
    (1, 3, 1): 0, (1, 3, 2): 0, (1, 3, 3): 0,
    (2, 1, 1): 1, (2, 1, 2): 0, (2, 1, 3): 0,
    (2, 2, 1): 0, (2, 2, 2): 1, (2, 2, 3): 0,
    (2, 3, 1): 0, (2, 3, 2): 0, (2, 3, 3): 1,
}

pijm = {
    (1, 1, 1): 3, (1, 1, 2): None, (1, 1, 3): None,
    (1, 2, 1): None, (1, 2, 2): None, (1, 2, 3): 4,
    (1, 3, 1): None, (1, 3, 2): None, (1, 3, 3): None,
    (2, 1, 1): 4, (2, 1, 2): None, (2, 1, 3): None,
    (2, 2, 1): None, (2, 2, 2): 2, (2, 2, 3): None,
    (2, 3, 1): None, (2, 3, 2): None, (2, 3, 3): 6,
}

STijm = {
    (1, 1, 1): 1, (1, 1, 2): None, (1, 1, 3): None,
    (1, 2, 1): None, (1, 2, 2): None, (1, 2, 3): 2,
    (1, 3, 1): None, (1, 3, 2): None, (1, 3, 3): None,
    (2, 1, 1): 2, (2, 1, 2): None, (2, 1, 3): None,
    (2, 2, 1): None, (2, 2, 2): 1, (2, 2, 3): None,
    (2, 3, 1): None, (2, 3, 2): None, (2, 3, 3): 1,
}

dmw = {
    (1, 1): 0, (1, 2): 0, (1, 3): 3
}

Q = 1

scenarios = [
    {"Scenario": 1, "Makespan": 20},
    {"Scenario": 2, "Makespan": 23},
    {"Scenario": 3, "Makespan": 29},
    {"Scenario": 4, "Makespan": 28},
    {"Scenario": 5, "Makespan": 25},
]

print("Input binary parameters fijm")
for key, value in fijm.items():
    print(f"Job {key[0]}, Operation {key[1]}, Machine {key[2]}: {value}")

print("\nInput parameters of processing time pijm")
for key, value in pijm.items():
    print(f"Job {key[0]}, Operation {key[1]}, Machine {key[2]}: {value}")

print("\nInput parameters of setup time STijm")
for key, value in STijm.items():
    print(f"Job {key[0]}, Operation {key[1]}, Machine {key[2]}: {value}")

print("\nInput parameters of maintenance duration dmw")
for key, value in dmw.items():
    print(f"Machine {key[0]}, Maintenance Activity {key[1]}: {value}")

print("\nMaximum number of maintenance activities")
print(f"Q: {Q}")

print("\nResults for each scenario")
for scenario in scenarios:
    print(f"Scenario {scenario['Scenario']}: Makespan {scenario['Makespan']}")


Input binary parameters fijm
Job 1, Operation 1, Machine 1: 1
Job 1, Operation 1, Machine 2: 0
Job 1, Operation 1, Machine 3: 0
Job 1, Operation 2, Machine 1: 0
Job 1, Operation 2, Machine 2: 0
Job 1, Operation 2, Machine 3: 1
Job 1, Operation 3, Machine 1: 0
Job 1, Operation 3, Machine 2: 0
Job 1, Operation 3, Machine 3: 0
Job 2, Operation 1, Machine 1: 1
Job 2, Operation 1, Machine 2: 0
Job 2, Operation 1, Machine 3: 0
Job 2, Operation 2, Machine 1: 0
Job 2, Operation 2, Machine 2: 1
Job 2, Operation 2, Machine 3: 0
Job 2, Operation 3, Machine 1: 0
Job 2, Operation 3, Machine 2: 0
Job 2, Operation 3, Machine 3: 1

Input parameters of processing time pijm
Job 1, Operation 1, Machine 1: 3
Job 1, Operation 1, Machine 2: None
Job 1, Operation 1, Machine 3: None
Job 1, Operation 2, Machine 1: None
Job 1, Operation 2, Machine 2: None
Job 1, Operation 2, Machine 3: 4
Job 1, Operation 3, Machine 1: None
Job 1, Operation 3, Machine 2: None
Job 1, Operation 3, Machine 3: None
Job 2, Operation 